In [1]:
from datetime import datetime
import fastf1
import pandas as pd


In [2]:

# Function to check if the latest race has been loaded
def has_latest_race_been_loaded(season, round_number):
    try:
        with open("loaded_races.txt", "r") as file:
            content = file.read()
            return f"{season}-{round_number}" in content
    except FileNotFoundError:
        return False


In [3]:

# Function to update the loaded races file
def update_loaded_races(season, round_number):
    with open("loaded_races.txt", "a") as file:
        file.write(f"{season}-{round_number}\n")


In [15]:
def extract_from_fastf1():
    # Enable cache
    fastf1.Cache.enable_cache('cache')

    # Get completed events for a specific season
    season = 2023
    schedule = fastf1.get_event_schedule(season)
    completed_events = [schedule.iloc[[event]] for event in schedule['RoundNumber'][1:] if schedule['EventDate'][event] < datetime.now()]

    # Get the latest completed event
    latest_event = completed_events[-1]
    round_number = latest_event['RoundNumber'].values[0]

    # Check if the latest race has been loaded
    if not has_latest_race_been_loaded(season, round_number):
        # Load the latest race data
        session_type = 'R'
        df_session = fastf1.get_session(season, round_number, session_type)
        df_session.load()

        print("Latest race data loaded")

        # Save the latest race data to a CSV file
        csv_filename = f"race_data_{season}_round_{round_number}.csv"
        df_session.results.to_csv(csv_filename, index=False)
        print(f"Latest race data saved to {csv_filename}")

        # Update the loaded races file
        update_loaded_races(season, round_number)

        return df_session.results

    else:
        print("Latest race has already been loaded")
        
        return -1


In [ ]:
extract_from_fastf1()

In [29]:
def get_fastest_laps_for_race(season, round_number):
    session_type = 'R'
    df_session = fastf1.get_session(season, round_number, session_type)
    df_session.load()
    
    # Group by driver and get the minimum lap time
    fastest_laps = df_session.laps.groupby('Driver')['LapTime'].min().reset_index()
    
    # Get the driver with the fastest lap time and their lap time
    fastest_driver = fastest_laps.loc[fastest_laps['LapTime'].idxmin()]
    
    # Create a dataframe with the fastest driver and their lap time
    df = pd.DataFrame({'driver': fastest_driver['Driver'], 'fastest_lap': fastest_driver['LapTime']}, index=[0])

    return df



In [31]:
print('Starting')
print(get_fastest_laps_for_race(2023, 3))
print('Finito')

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data


Starting


c:\Users\vlamf\Coding\anaconda3\lib\site-packages\fastf1\core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
c:\Users\vlamf\Coding\anaconda3\lib\site-packages\fastf1\core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
c:\Users\vlamf\Coding\anaconda3\lib\site-packages\fastf1\core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
c:\Users\vlamf\Coding\anaconda3\lib\site-packages\fastf1\core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a futu

  driver            fastest_lap
0    PER 0 days 00:01:20.235000
Finito
